<a href="https://githubtocolab.com/geonextgis/geeagri/blob/main/docs/examples/principal_component_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install the latest version of [geeagri](https://geonextgis.github.io/geeagri) if needed.

In [ ]:
# !pip install -U geeagri

## Import libraries

In [ ]:
import ee
import geemap
from geeagri.analysis import PCA
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["font.family"] = "DeJavu Serif"
plt.rcParams["font.serif"] = "Times New Roman"

## Initialize a Map object
Authenticate and initialize Earth Engine. If it doesn't work, specify a project name

In [ ]:
# ee.Authenticate()
# ee.Initialize(project='your-project-id')

Map = geemap.Map(basemap="SATELLITE")
Map

## Import region of interest

In [ ]:
bbox = [-120.398369, 37.006574, -120.208168, 37.101924]
region = ee.Geometry.BBox(*bbox)
region_style = {"color": "red", "width": 1}
Map.addLayer(region, region_style, "Region")
Map.centerObject(region, 12)

## Load Sentinel-2 data and create a median composite

In [ ]:
image = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(region)
    .filterDate("2024-01-01", "2025-01-01")
    .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 10)
    .select(["B.*"])
    .median()
    .multiply(0.0001)
    .clip(region)
)

image_vis = {"bands": ["B8", "B4", "B3"], "min": 0, "max": 0.3}

Map.addLayer(image, image_vis, "Sentinel 2 FCC")

## Principal Component Analysis

In [ ]:
pca = PCA(image, region=region, scale=100)
pcs = pca.get_principal_components()

# Define visualization parameters
pca_vis = {
    "bands": ["pc1", "pc2", "pc3"],
    "min": -2,
    "max": 2,
}

# Add PCA layer
Map.addLayer(pcs, pca_vis, "PCA RGB Composite")

## Plot the explained variance by principal components

In [ ]:
# Plot the explained variance by principle components
explained_variance = pca.get_explained_variance()

sns.barplot(
    data=explained_variance,
    x="pc",
    y="variance_explained",
    hue="variance_explained",
    palette="Greens",
    edgecolor="k",
)
plt.show()